In [76]:
import pandas as pd
import numpy as np
import sqlite3

def read(file):
  data = pd.read_csv(file)

  print("CSV file has Columns \n{}".format(data.columns))
  print("\nThe CSV file has data types \n{}".format(data.dtypes))
  print(f"\nThere are {data.shape[0]} rows and {data.shape[1]} columns in this Data frame currently")

  return data
  
data_file = read("DataAnalystTask.csv")

CSV file has Columns 
Index(['Purchase__basketValueGross', 'Purchase__purchaseType',
       'Purchase__overallBasketSavings', 'Purchase__storeId',
       'Purchase__paymentType__category', 'Purchase__paymentType__amount',
       'Purchase__timeStamp', 'Purchase__basketValueNet', 'Purchase__says',
       'Purchase__storeName', 'Purchase__storeFormat',
       'Purchase__product__name', 'Purchase__product__quantity',
       'Purchase__product__channel', 'Purchase__product__price'],
      dtype='object')

The CSV file has data types 
Purchase__basketValueGross          object
Purchase__purchaseType              object
Purchase__overallBasketSavings     float64
Purchase__storeId                  float64
Purchase__paymentType__category    float64
Purchase__paymentType__amount      float64
Purchase__timeStamp                 object
Purchase__basketValueNet           float64
Purchase__says                     float64
Purchase__storeName                 object
Purchase__storeFormat             

In [77]:
## Transforming the data
## Clean up data, trim columns, filter and order it
## Store name and timestamp have MNAR data
## Gathering columns which are needed

## This dataset has 430 unnamed stores which have been filled with Undesignated store and empty purchased timestamps which has been filled with the median of the dataset for timestamps
## I have removed any data without any product names as well 
from datetime import timedelta
import pandas as pd
import numpy as np

# parsing duration as a timedelta
def parse_duration(x):
    try:
        if pd.isna(x): return np.nan
        mins, secs = x.split(':')
        return timedelta(minutes=int(mins), seconds=float(secs))
    except:
        return np.nan

def transform(data_file):
    required_data = data_file[["Purchase__product__name","Purchase__product__price","Purchase__timeStamp","Purchase__product__quantity","Purchase__storeName"]]

    ## removing products without a product name (NaN)
    required_data = required_data.dropna(subset=["Purchase__product__name"])


    #Adjusting the error in timestamp
    required_data['Purchase__timeStamp'] = required_data['Purchase__timeStamp'].apply(parse_duration)
    #getting the median excluding NAN
    median_time = required_data["Purchase__timeStamp"].median()
    #filling the null values with median
    required_data["Purchase__timeStamp"] = required_data["Purchase__timeStamp"].fillna(median_time)

    df = pd.DataFrame(required_data)
    #debug to find highest frequency of store names
    #display(df.groupby('Purchase__storeName').count())

    #Filling missing store names with store 'Undesignated store'
    df[["Purchase__storeName"]] = df[["Purchase__storeName"]].fillna("Undesignated store")
    display(df)

    return df

  

In [78]:
## Loading the dataframe into a database
import pandas as pd
import sqlite3

def load(dataframe, database_name, table_name):
  #create the connection obj
  con = sqlite3.connect(database_name)

  #now we will write the data to a specific table which we have specified in the param
  dataframe.to_sql(name=table_name, con=con, if_exists="replace", index=False)
  print("Dataframe has been loaded to sqlite\n")

  # reading the data which has been loaded for validation
  loaded_df = pd.read_sql(sql=f"SELECT * FROM {table_name}", con=con)

  try:
      #ensuring shape of DF loaded is same as what we have from the transformation stage
      assert dataframe.shape == loaded_df.shape
      print(f"Data in {table_name} table has been loaded successfuly")

  except AssertionError:
     print(f"Error! Data in {table_name} table has not been loaded successfuly")

#calling function
load(dataframe = df, database_name="Data_Pipeline", table_name="Store_Purchase")

Dataframe has been loaded to sqlite

Data in Store_Purchase table has been loaded successfuly


/var/folders/0n/gy8jzz9n1yx5nmwnqmqyckbc0000gn/T/ipykernel_91918/1149635945.py:10: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  dataframe.to_sql(name=table_name, con=con, if_exists="replace", index=False)


In [79]:
import pandas as pd
import sqlite3
#Extract data
data_file = read("DataAnalystTask.csv")

CSV file has Columns 
Index(['Purchase__basketValueGross', 'Purchase__purchaseType',
       'Purchase__overallBasketSavings', 'Purchase__storeId',
       'Purchase__paymentType__category', 'Purchase__paymentType__amount',
       'Purchase__timeStamp', 'Purchase__basketValueNet', 'Purchase__says',
       'Purchase__storeName', 'Purchase__storeFormat',
       'Purchase__product__name', 'Purchase__product__quantity',
       'Purchase__product__channel', 'Purchase__product__price'],
      dtype='object')

The CSV file has data types 
Purchase__basketValueGross          object
Purchase__purchaseType              object
Purchase__overallBasketSavings     float64
Purchase__storeId                  float64
Purchase__paymentType__category    float64
Purchase__paymentType__amount      float64
Purchase__timeStamp                 object
Purchase__basketValueNet           float64
Purchase__says                     float64
Purchase__storeName                 object
Purchase__storeFormat             

In [80]:
#transform the data
required_data = transform(data_file)

,Purchase__product__name,Purchase__product__price,Purchase__timeStamp,Purchase__product__quantity,Purchase__storeName
0,Koko Dairy Free Unsweetened Alternative Longli...,1.50,0 days 00:37:36.800000,5.0,SOUTHWARK
1,Tesco Bunched Spring Onions 100G,0.37,0 days 00:26:06.100000,1.0,Undesignated store
2,Tesco Finest Cheesecake Passion Fruit & Raspbe...,4.00,0 days 00:26:06.100000,1.0,Undesignated store
3,Tesco Mixed Sized Organic Eggs 6 Pack,1.80,0 days 00:26:06.100000,1.0,Undesignated store
4,Vanish Pet Expert Carpet Care Foam 600Ml,6.00,0 days 00:26:06.100000,1.0,Undesignated store
...,...,...,...,...,...
438,Tesco Clementine Or Sweet Easy Peeler Pack 600G,1.35,0 days 00:26:06.100000,1.0,Undesignated store
439,Tesco Oaty Rounds Biscuits 300G,0.45,0 days 00:26:06.100000,1.0,Undesignated store
440,Kraft Philadelphia Cheese 200 G Tub,1.95,0 days 00:26:06.100000,2.0,Undesignated store
441,Tesco Garlic Baguette 205G,0.90,0 days 00:50:31.100000,1.0,SOUTHWARK


In [81]:
#load the data
load(dataframe = required_data, database_name="Data_Pipeline", table_name="Store_Purchase")

Dataframe has been loaded to sqlite

Data in Store_Purchase table has been loaded successfuly


/var/folders/0n/gy8jzz9n1yx5nmwnqmqyckbc0000gn/T/ipykernel_91918/1149635945.py:10: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  dataframe.to_sql(name=table_name, con=con, if_exists="replace", index=False)
